<a href="https://colab.research.google.com/github/yexf308/AppliedStochasticProcess/blob/main/HW/HW5/560Sp22HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$
# Homework 5
## Homework guideline
- The deadline is May 5th 10:30am. Submission after the deadline will not be graded. 


- Submit your work(your reasoning and your code) as a SINGLE .ipynb document. Please rename the document as _HW1_YOURNAME.ipynb_ (for example, _HW1_FELIX.ipynb_). You are responsible for checking that you have correctly submitted the correct document. If your code cannot run, you may receive NO point. 




- You only use the Python packages included in the following cell. You are not allowed to use other advanced package or modules unless you are permitted to. 

- In your final submission include the plots produced by the unedited code as presented below, as well as any additional plots produced after editing the code during the course of a problem. You may find it necessary to copy/paste relevant code into additional cells to accomplish this.

- Feel free to use the lecture notes and other resources. But you
must understand, write, and hand in your own answers. In addition, you must write and submit
your own code in the programming part of the assignment (we may run your code).
If you copy someone else homework solution, both of you may receive ZERO point. 


- Colab is preferred. However, if you use Anaconda, please download the .mat file locally and save it to the same folder as this homework file. 



# Q1: Implement forward filtering backward sampling (50pt)
We now consider the sampling of latent states $h$ given an observation $v$. Firstly, it is important to differentiate sampling from Viterbi-backtracking:
* Viterbi-backtracking: $h_{1:n}^* = \text{argmax } p(h_{1:n} | v_{1:n})$
    * There is only one $h_{1:n}^*$ that has the largest probability (under the assumption of a unique global maximum).
* Sampling: $h_{1:n} \sim p(h_{1:n} | v_{1:n})$
    * In sampling we attempt to generate several latent sequences $h_{1:n}$, not just the most probable.
    * Note that the sequence with $h_{1:n}^*$ still has the largest probability to get sampled.


It is also helpful to view sampling as a random function (as opposed to a deterministic function): 
* Viterbi-backtracking is a deterministic function since it always gives the same output.
* Sampling is a random function since each time it may give us different outputs.

Sampling from the posterior $p(h_{1:n} | v_{1:n})$ can be done with backward-sampling:
\begin{align}
    h_n &\sim p(h_n | v_{1:n}) \\ 
    h_{t-1} &\sim p(h_{t-1} | h_t, v_{1:n}) \qquad \text{for }t\le n
\end{align}

In the previous Homework, we showed that 
\begin{align}
p(h_{t-1}| h_t, v_{1:n}) =\frac{\alpha(h_{t-1})}{\alpha(h_t)}p(h_t| h_{t-1})p(v_t|h_t)
\end{align}
Since the probabilities above are expressed as a function of the transition, emission, and the alpha variables, we will use the forward algorithm to obtain them.

## Algorithm
We thus obtain the following algorithm to generate samples from $p(h_1, \dots, h_n | v_{1:n})$:

- Run the filtering to determine all $\alpha(h_t)$ forward in time for $t=1, \dots, n$.

- Sample $h_n$ from $p(h_n|v_{1:n})\propto\alpha(h_n)$.

-  Go backwards in time using
\begin{align}
p(h_{t-1}| h_t, v_{1:n}) =\frac{\alpha(h_{t-1})}{\alpha(h_t)}p(h_t| h_{t-1})p(v_t|h_t)
\end{align}
to generate samples $h_{t-1}| h_t, v_{1:n}$ for $t=n,\dots, 2$.

You will implement this algorithm in the following function. 
The computation of the equations above are performed in the log domain, hence

\begin{align*}
\log p(h_{t - 1}| h_t, v_{1:T}) &=  \log \alpha(h_{t-1}) + \log p(h_t | h_{t-1}) + \log p(v_t | h_t) - \log \alpha(h_t)
\end{align*}


In [ ]:
def sampling(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T
        
    Returns:
        h: sampled sequence, a vector of length T
    """
    return

## Verify the algorthm with examples
### Test case 1
we call the sampling function to see its output. Try running it multiple times to see that the outputs can be different.

In [ ]:
log_initial = np.log(np.array([0.3, 0.7]))
log_transition = np.log(np.array([[0.2, 0.8], 
                                  [0.6, 0.4]]))
log_emission = np.log(np.array([[0.6, 0.7], 
                                [0.4, 0.3]]))
v = [0, 0]
p_h, h = sampling(log_initial, log_transition, log_emission, v)
print(np.prod(p_h), h)


### Test case 2
We verify our implementation by the law of large numbers: if we call the sampling function many times, then the frequency of each sampled latent sequence will be approximately equal to their probability. 

Note that the frequency of the latent sequence [1, 0] returned from the simulation should close to 0.4045, i.e., approximately same as the true probability returned by the Viterbi algorithm. You may increase the number `N=100000`, which should return an even closer result.

In [ ]:
h_freq = {'00': 0, '01': 0, '10': 0, '11': 0}
N = 10000
for _ in range(N):
    p_h, h = sampling(log_initial, log_transition, log_emission, v)
    h_freq[f'{h[0]}{h[1]}'] += 1

for h in h_freq:
    print('latent %s freq %.4f' % (h, h_freq[h] / N))

# Q2: (optional) Correction to your previous homework question
You may pick any one question in homework 1-4 that didn't perform well, and now you have the chance to correct your mistakes. If you successfully correct your mistakes, your previous grade will be replaced by the current score.

State Your question that you want to correct:

In [ ]:
# Your new code starts here

# Your New Solution: